In [0]:
!pip install kaggle

In [0]:
import os

os.environ["KAGGLE_USERNAME"] = "dataengineer25"
os.environ["KAGGLE_KEY"] = "KGAT_1c6a6e6d91507ed911360a950c2a0946"

print("Kaggle credentials configured!")

In [0]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce
""")

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")

In [0]:
!kaggle datasets download \
  -d mkechinov/ecommerce-behavior-data-from-multi-category-store \
  -p /Volumes/workspace/ecommerce/ecommerce_data \
  --unzip


In [0]:
%restart_python

In [0]:
df_n = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv")

In [0]:
df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")


In [0]:
df_n.printSchema()

In [0]:
# Explicitly define schema
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, LongType, DoubleType

schema = StructType([
    StructField("event_time", TimestampType(), True),
    StructField("event_type", StringType(), True),
    StructField("product_id", LongType(), True),
    StructField("category_id", LongType(), True),
    StructField("category_code", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("user_id", LongType(), True),
    StructField("user_session", StringType(), True)
])

events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",
                        header=True, schema=schema)

events.printSchema()
events.show()
events.count()
df.count()
df_n.count()
df_n.show()
df.show()
df.printSchema()
df_n.printSchema()



In [0]:
df_n.describe().show()

In [0]:
df.describe().show()

In [0]:
def load_ecommerce_data(month, sample_fraction=None):
  """
  Load the ecommerce data for the given month .
  """
  print(f"Loading data for month: {month}")
  df = spark.read.csv(f"/Volumes/workspace/ecommerce/ecommerce_data/2019-{month}.csv")
  if sample_fraction is not None:
    df = df.sample(sample_fraction)
  return df
# Use sampling for testing
sampled = load_ecommerce_data("Oct", sample_fraction=0.1)
print(f"Sampled data: {sampled.count():,} rows")